In [ ]:
#ライブラリの自動リロード
%load_ext autoreload
%autoreload 2

from src.GGUFBot import GGUFBot
from src.HFDataset import HFDataset
from src.SimpleQuestionGenerator import SimpleQuestionGenerator
from src.AnswerGenerator import AnswerGenerator
import pandas as pd
import random
import json

In [ ]:
from datasets import load_dataset
dataset = load_dataset("minnade/chat-daily",split="train")
dataset=dataset.filter(lambda x: x["role"] == "user")

questions=[]
for d in dataset:
    questions.append(d["body"])

In [ ]:

n_layers=30
max_new_tokens=4000
model_path="/home/hatakeyama/python/ChatServer/model/Mixtral-8x22B-Instruct-v0.1.Q5_K_M-00001-of-00004.gguf"
bot=GGUFBot(model_path,max_new_tokens=max_new_tokens,n_ctx=max_new_tokens,n_gpu_layers=n_layers)

In [ ]:

a_gen = AnswerGenerator(bot,n_answers=2)
out_path="data/augmented_q_and_a.jsonl"

In [5]:
question=random.choice(questions)
question_template = f"""以下の問題・質問・指示の類題を日本語で作成してください。
・類題はもとの問題からは必ず情報を追加・修正・削除し、内容、形式、記述方式が全く異なるようにすること
・作成した内容のみを出力すること
"""
while True:
    q=question_template+question
    try:
        new_question=bot.ask(q).replace("#類題","").replace("#問題","").strip()[:3000]
        record={"question":new_question}

        a_gen(record)
        with open(out_path, 'a') as f:
            f.write(json.dumps(record,ensure_ascii=False)+"\n")
    except:
        pass
    question=new_question
    #question=question[:random.randint(0,len(question))]
    if random.randint(0,3)==0:
        question=question[:int(len(question)*0.8)]
    elif random.randint(0,3)==1:
        question=question[int(len(question)*0.2):]
    if random.randint(0,3)==0:
        question=random.choice(questions)




llama_print_timings:        load time =   10998.73 ms
llama_print_timings:      sample time =     355.51 ms /  1513 runs   (    0.23 ms per token,  4255.91 tokens per second)
llama_print_timings: prompt eval time =   21797.48 ms /   940 tokens (   23.19 ms per token,    43.12 tokens per second)
llama_print_timings:        eval time =  424020.26 ms /  1512 runs   (  280.44 ms per token,     3.57 tokens per second)
llama_print_timings:       total time =  454194.88 ms /  2452 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   10998.73 ms
llama_print_timings:      sample time =      71.74 ms /   290 runs   (    0.25 ms per token,  4042.15 tokens per second)
llama_print_timings: prompt eval time =   34714.65 ms /  1526 tokens (   22.75 ms per token,    43.96 tokens per second)
llama_print_timings:        eval time =   79615.77 ms /   289 runs   (  275.49 ms per token,     3.63 tokens per second)
llama_print_timings:       total time =  115129.32 ms /  1815 